In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

In [5]:
train = pd.read_csv("./train.txt", sep=";")
train.columns = ["text","sentiment"]
test = pd.read_csv("./test.txt", sep=";")
test.columns = ["text","sentiment"]
train.head()

,text,sentiment
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [7]:
train_text = train['text']
test_text = test['text']
if len(train_text) == len(test_text):
    print("is equal")
else:
    print("not equal")
    print("train values length: " + str(len(train_text)) + ", and test values length: " + str(len(test_text)))


not equal
train values length: 15999, and test values length: 1999


In [10]:
#trim Train length
train_to_match_test = train[:1999]